In [1]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRFClassifier
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv").dropna()
df = df.reset_index(drop=True)

In [3]:
importances = []
results = []

In [4]:
df['h_result'] = df['score1'] - df['score2']
df['h_importance'] = df['importance1'] - df['importance2']

In [5]:
for x in df['h_result']:
  if x > 0:
    results.append("HOME")
  elif x < 0:
    results.append("AWAY")
  else:
    results.append("DRAW")

In [6]:
for y in df['h_importance']:
  if y > 0:
    importances.append("HOME")
  elif y < 0:
    importances.append("AWAY")
  else:
    importances.append("DRAW")

In [8]:
df = df.drop(columns=['h_result','h_importance'])

In [9]:
df['importance'] = importances
df['result'] = results

In [10]:
matches = []

In [11]:
for z in range(len(df)):
  i = df['importance'][z]
  r = df['result'][z]
  if i == r:
    matches.append("MATCH")
  else:
    matches.append("NO MATCH")

In [12]:
df['importance_match'] = matches

In [13]:
X_final = df[['score1','score2','importance1','importance2']]
y_final = df['importance_match']

In [14]:
accuracy_values = []

In [15]:
def model():
  count = 0
  while count < 30:
    X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=count)
    model = XGBRFClassifier(max_depth=6, eval_metric='mlogloss', objective='binary:logistic', n_estimators=400).fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    accuracy_values.append(accuracy)
    count += 1

In [16]:
model = model()

In [17]:
def round_3(num):
  num = np.round(num, 3)
  num = num * 100
  return num

In [19]:
acc_min = round_3(np.min(accuracy_values))
acc_med = round_3(np.median(accuracy_values))
acc_mean = round_3(np.mean(accuracy_values))
acc_max = round_3(np.max(accuracy_values))
print("Accuracy Values: Minimum {}% | Median: {}% | Mean: {}% | Maximum: {}%".format(acc_min, acc_med, acc_mean, acc_max))

Accuracy Values: Minimum 85.7% | Median: 87.2% | Mean: 87.2% | Maximum: 88.7%
